In [31]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

legislatura = 'Legislatura XLVIII'

legisladores_df = pd.read_csv('data/%s/legisladores.csv' % legislatura)
legisladores_df.head()

,cuerpo,email,id_legislador,lema,nombre
0,NaN,eagazzi@parlamento.gub.uy,5146,NaN,"Agazzi, Ernesto"
1,NaN,NaN,2743,NaN,"Alcorta, Ricardo"
2,NaN,secretaria.ministro@mrree.gub.uy,8883,NaN,"Almagro Lemes, Luis Leonardo"
3,SENADO,valonso@parlamento.gub.uy,9891,PARTIDO NACIONAL,"Alonso, Verónica"
4,NaN,nolivera@parlamento.gub.uy,11535,NaN,"Álvarez López, María Dolores"


## Dimensiones

### Asistencia
Presiencia en el plenario y comisiones<br>
Promedio de la presencia en el plenario y las comisiones (asistencias / citaciones)

In [32]:
asistencia_plenario_df = pd.read_csv('data/%s/asistencia_plenario.csv' % legislatura)
asistencia_comision_df = pd.read_csv('data/%s/asistencia_comisiones.csv' % legislatura)
asistencia_df = pd.concat([asistencia_plenario_df[['id_legislador', 'asistencias', 'citaciones']], asistencia_comision_df[['id_legislador', 'asistencias', 'citaciones']]])
asistencia_df = asistencia_df.groupby('id_legislador').sum()

# Cálculo de z-score de asistencias
asistencia_df.loc[:,'asistencia_med'] = asistencia_df.apply(lambda x : x['asistencias'] * 1.0 / x['citaciones'], axis=1)
asistencia_media = asistencia_df['asistencia_med'].mean()
asistencia_std = asistencia_df['asistencia_med'].std()
asistencia_df.loc[:,'asistencia_zidx'] = asistencia_df.apply(lambda x : (x['asistencia_med'] * 1.0 - asistencia_media) / asistencia_std, axis=1)
asistencia_df.reset_index(inplace=True)
asistencia_df = asistencia_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
asistencia_df.head()

,id_legislador,asistencias,citaciones,asistencia_med,asistencia_zidx,nombre,cuerpo,lema
0,149,383,421,0.909739,0.195146,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,346,448,0.772321,-0.623214,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,502,597,0.840871,-0.214981,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,316,412,0.766990,-0.654962,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,359,439,0.817768,-0.352568,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Proyectos
Incluye las siguientes actividades:
* Presentación o firma de proyectos de ley
* Proyectos de resolución
* Proyectos de minutas
* Proyectos de declaración de cuerpo
<br><br>El índice se basa en la suma de todos los proyectos presentados

In [33]:
proyectos_presentados_df = pd.read_csv('data/%s/proyectos_presentados.csv' % legislatura)
proyectos_df = pd.read_csv('data/%s/proyectos.csv' % legislatura)
proyectos_presentados_df = proyectos_presentados_df.merge(proyectos_df, on='id_proyecto', how='left')

proyectos_presentados_df = proyectos_presentados_df[proyectos_presentados_df['tipo'].isin(['PROYECTO DE DECLARACION', 'PROYECTO DE LEY', 'PROYECTO DE MINUTA DE COMUNICACION','PROYECTO DE RESOLUCION'])]
proyectos_presentados_df = proyectos_presentados_df[['id_legislador', 'id_proyecto']].groupby('id_legislador').size().reset_index(name='proyectos_total')

proyectos_presentados_media = proyectos_presentados_df['proyectos_total'].mean()
proyectos_presentados_std = proyectos_presentados_df['proyectos_total'].std()
proyectos_presentados_df.loc[:,'proyectos_presentados_zidx'] = proyectos_presentados_df.apply(lambda x : (x['proyectos_total'] * 1.0 - proyectos_presentados_media) / proyectos_presentados_std, axis=1)
proyectos_presentados_df = proyectos_presentados_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
proyectos_presentados_df.head()

,id_legislador,proyectos_total,proyectos_presentados_zidx,nombre,cuerpo,lema
0,149,23,1.866684,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,19,1.355071,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,24,1.994588,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,28,2.506201,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,14,0.715554,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Informante en comisión
Informantes de proyectos en las comisiones en las que participa<br>
El índice se basa en la suma de número de acciones

In [34]:
actuacion_parlamentaria_df = pd.read_csv('data/%s/actuacion_parlamentaria.csv' % legislatura, index_col=0)
informante_comision_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INFORMA']
informante_comision_df = informante_comision_df.groupby('id_legislador').size().reset_index(name='informes_total')
informante_comision_media = informante_comision_df['informes_total'].mean()
informante_comision_std = informante_comision_df['informes_total'].std()
informante_comision_df.loc[:,'informe_comision_zidx'] = informante_comision_df.apply(lambda x : (x['informes_total'] * 1.0 - informante_comision_media) / informante_comision_std, axis=1)
informante_comision_df = informante_comision_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
informante_comision_df.head()


,id_legislador,informes_total,informe_comision_zidx,nombre,cuerpo,lema
0,149,9,0.069247,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,1,-0.576293,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,27,1.521711,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,9,0.069247,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,13,0.392017,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Debate
Interviene en debates sobre proyectos<br>
El índice se basa en la suma de las intervenciones en el plenario

In [35]:
interviene_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INTERVIENE']
interviene_df = interviene_df.groupby('id_legislador').size().reset_index(name='interveciones_total')
interviene_media = interviene_df['interveciones_total'].mean()
interviene_std = interviene_df['interveciones_total'].std()
interviene_df.loc[:,'intervenciones_zidx'] = interviene_df.apply(lambda x : (x['interveciones_total'] * 1.0 - interviene_media) / interviene_std, axis=1)
interviene_df = interviene_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
interviene_df.head()


,id_legislador,interveciones_total,intervenciones_zidx,nombre,cuerpo,lema
0,149,290,3.283557,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,131,1.167094,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,324,3.736135,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,230,2.484892,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,117,0.980738,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Exposiciones
Escribe notas o realiza exposiciones sobre temas determinados.<br>
El índice se basa en la suma de las exposiciones realizadas

In [36]:
expone_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'EXPONE']
expone_df = expone_df.groupby('id_legislador').size().reset_index(name='exposiciones_total')
expone_media = expone_df['exposiciones_total'].mean()
expone_std = expone_df['exposiciones_total'].std()
expone_df.loc[:,'exposiciones_zidx'] = expone_df.apply(lambda x : (x['exposiciones_total'] * 1.0 - expone_media) / expone_std, axis=1)
expone_df = expone_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
expone_df.head()


,id_legislador,exposiciones_total,exposiciones_zidx,nombre,cuerpo,lema
0,149,24,0.747609,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,22,0.627817,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,23,0.687713,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,5,-0.390418,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,19,0.448128,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Pedidos de informe
Solicita un pedido de informes al gobierno.<br>
El índice se basa en la suma de pedidos de informe

In [37]:
pedidos_de_informe_df = pd.read_csv('data/%s/pedidos_informe.csv' % legislatura, index_col=0)
pedidos_de_informe_df = pedidos_de_informe_df[['id_legislador', 'id_proyecto']].groupby('id_legislador').size().reset_index(name='pedidos_informe_total')
pedidos_de_informe_media = pedidos_de_informe_df['pedidos_informe_total'].mean()
pedidos_de_informe_std = pedidos_de_informe_df['pedidos_informe_total'].std()
pedidos_de_informe_df.loc[:,'pedidos_informe_zidx'] = pedidos_de_informe_df.apply(lambda x : (x['pedidos_informe_total'] * 1.0 - pedidos_de_informe_media) / pedidos_de_informe_std, axis=1)
pedidos_de_informe_df = pedidos_de_informe_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
pedidos_de_informe_df.head()


,id_legislador,pedidos_informe_total,pedidos_informe_zidx,nombre,cuerpo,lema
0,149,33,1.274873,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,14,0.032331,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,7,-0.425447,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,14,0.032331,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,2,-0.752432,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


In [38]:
indice_senadores_df = asistencia_df[['nombre', 'cuerpo', 'lema', 'id_legislador', 'asistencia_zidx']].merge(proyectos_presentados_df[['id_legislador', 'proyectos_presentados_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(informante_comision_df[['id_legislador', 'informe_comision_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(interviene_df[['id_legislador', 'intervenciones_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(expone_df[['id_legislador', 'exposiciones_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(pedidos_de_informe_df[['id_legislador', 'pedidos_informe_zidx']], on='id_legislador')
indice_senadores_df.loc[:, 'zidx'] = indice_senadores_df['asistencia_zidx'] +indice_senadores_df['asistencia_zidx'] + indice_senadores_df['proyectos_presentados_zidx'] +indice_senadores_df['informe_comision_zidx'] + indice_senadores_df['intervenciones_zidx'] + indice_senadores_df['exposiciones_zidx'] + indice_senadores_df['pedidos_informe_zidx']
indice_senadores_df.sort_values('zidx', ascending=False).head()


,nombre,cuerpo,lema,id_legislador,asistencia_zidx,proyectos_presentados_zidx,informe_comision_zidx,intervenciones_zidx,exposiciones_zidx,pedidos_informe_zidx,zidx
12,"Bordaberry Herrán, Juan Pedro",SENADO,PARTIDO COLORADO,922,0.143693,1.738781,1.925174,7.596352,2.245014,2.190430,15.983136
21,"Abdala, Pablo",REPRESENTANTE,PARTIDO NACIONAL,3062,-0.016844,2.889912,0.149939,4.934133,2.304911,1.732651,11.977858
7,"Martínez Huelmo, Rubén",SENADO,PARTIDO FRENTE AMPLIO,408,0.471981,0.587651,6.847415,2.604692,0.567921,-0.752432,10.799207
63,"Arocena Argul, José Andrés",REPRESENTANTE,PARTIDO NACIONAL,9953,-1.094313,2.889912,-0.172831,0.834316,7.216399,1.667254,10.246424
5,"Mieres, Pablo",SENADO,PARTIDO INDEPENDIENTE,327,-0.012960,1.738781,0.553402,4.601356,0.987194,1.863445,9.718258


In [39]:
actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'].isnull()].head()

,fecha,id_legislador,tipo
detalle,,,
Presenta el proyecto de resolución. Carpeta: CSS/180/2015; SEGURIDAD PÚBLICA Y CONVIVENCIA. COMISIÓN ESPECIAL. DESIGNACIÓN.,2015-04-07 00:00:00,5146,NaN
Presenta el proyecto de declaración. Carpeta: CSS/182/2015; UNIVERSIDAD GARISSA. ATENTADO. VÍCTIMAS. SOLIDARIDAD. DECLARACIÓN.,2015-04-07 00:00:00,5146,NaN
Solicita la realización de sesión de la Cámara de Senadores en régimen de Comisión General invitando a l Sr. Danilo Astori (MINISTRO DE ECONOMIA Y FINANZAS - MEF*) CSS/181/2015; ECONOMÍA NACIONAL. PERSPECTIVAS Y ESTADO ACTUAL. INFORME. MINISTRO ECONOMÍA Y FINANZAS. LLAMADO A SALA.,2015-04-07 00:00:00,5146,NaN
"Presenta el proyecto de resolución. Carpeta: CSS/190/2015; GALEANO, Eduardo. FALLECIMIENTO. HOMENAJE. Se da cuenta al cuerpo: tomo 534 pág. 235 d.s. 9",2015-04-14 00:00:00,5146,NaN
"Presenta el proyecto de resolución. Asunto: 125032; GALEANO, EDUARDO. FALLECIMIENTO. HOMENAJE.",2015-04-15 00:00:00,5146,NaN


In [40]:
df = pd.read_csv('data/%s/convocatoria.csv' % legislatura, infer_datetime_format=True, parse_dates=['fecha_ini', 'fecha_fin'])
convocatoria_df = df.merge(legisladores_df[['id_legislador', 'nombre']], on='id_legislador', how='left')
convocatoria_df.head()

,camara,departamento,duracion_segundos,fecha_fin,fecha_ini,id_legislador,lema,sublema,titular,nombre
0,Cámara de Senadores,NaN,0.0,2016-12-14 00:00:00,2015-02-15,5146,NaN,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,NaN,"Agazzi, Ernesto"
1,Cámara de Senadores,NaN,86340.0,2015-03-18 23:59:00,2015-03-18,2743,NaN,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,NaN,"Alcorta, Ricardo"
2,Cámara de Senadores,NaN,86340.0,2015-04-08 23:59:00,2015-04-07,2743,PARTIDO FRENTE AMPLIO,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,"Ayala, Patricia","Alcorta, Ricardo"
3,Cámara de Senadores,NaN,86340.0,2015-04-09 23:59:00,2015-04-09,2743,PARTIDO FRENTE AMPLIO,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,"Ayala, Patricia","Alcorta, Ricardo"
4,Cámara de Senadores,NaN,86340.0,2015-04-16 23:59:00,2015-04-14,2743,PARTIDO FRENTE AMPLIO,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,"Ayala, Patricia","Alcorta, Ricardo"


In [42]:
convocatoria_df[['nombre', 'duracion_segundos']].groupby('nombre').sum()

,duracion_segundos
nombre,
"Abdala, Pablo",0.0
"Abreu Castillo, Aníbal",662040.0
"Acosta, Auro",5266740.0
"Acosta, José Luis",1554120.0
"Agazzi, Ernesto",474900.0
"Agüero Demetrio, Nelson Leodato",2072160.0
"Alaluf, Rosario",1295100.0
"Alcorta, Ricardo",1928700.0
"Algorta Brit, Felipe José",259020.0


In [ ]:
print(convocatoria_df.columns)
convocatoria_df[['titular']].drop_duplicates()